In [1]:
import pandas as pd
import dask.dataframe as dd

import ast

import os
from dotenv import load_dotenv
from warnings import filterwarnings

In [2]:
load_dotenv()
filterwarnings("ignore")

CLOUD_STORAGE_BUCKET = os.getenv("CLOUD_STORAGE_BUCKET")
CLOUD_STORAGE_BUCKET

'gs://graph_data_science/twitter'

## Ties

In [3]:
data_type = "ties"
data = dd.read_csv(f"{CLOUD_STORAGE_BUCKET}/data/{data_type}*.csv").compute()
# data.following = data.following.apply(ast.literal_eval)
# data = data.explode("following").dropna()
NEW_CLOUD_STORAGE_BUCKET = f'gs://ego_networks/twitter/{data_type}'

print(data.shape)
print(data.user.nunique(), data.following.nunique())
data.head()

(243, 2)
231 233


,user,following
0,1210565171107815429,"[1281443510, 632966383, 27631809, 14922097, 22..."
1,3432386097,"[33836629, 188343397, 235684766, 17268874, 439..."
2,1120633726478823425,[]
3,4398626122,[]
4,48008938,"[1464797992872034304, 630995607, 27493883, 123..."


In [4]:
data.user.value_counts()

1427408443296657408    12
1120633726478823425     2
1210565171107815429     1
1075605139              1
14717311                1
                       ..
842780484752949253      1
3429950987              1
84022193                1
16047950                1
25422302                1
Name: user, Length: 231, dtype: int64

In [5]:
data_n = data[~(data.user.isin([1427408443296657408, 1120633726478823425]))]

In [6]:
dd.to_parquet(dd.from_pandas(data_n, npartitions = 10), 
              path = NEW_CLOUD_STORAGE_BUCKET,
              append = False,
              overwrite = True,
             ignore_divisions=True)

# during append operations
# dd.to_parquet(dd.from_pandas(data, npartitions = 10), 
#               path = NEW_CLOUD_STORAGE_BUCKET,
#               append = True,
#               overwrite = False,
#              ignore_divisions=True)

data_n = dd.read_parquet(
              path = NEW_CLOUD_STORAGE_BUCKET).compute()
data_n.head()

,user,following
0,1210565171107815429,"[1281443510, 632966383, 27631809, 14922097, 22..."
0,1105508110213545984,"[973641576806305794, 1174622749, 9827789439493..."
0,1499503033859805184,"[1492967790587777027, 777902325771988992, 8401..."
0,859816394556284929,"[895332160130891776, 1654188770, 13298072, 347..."
1,3432386097,"[33836629, 188343397, 235684766, 17268874, 439..."


## Node Features

In [15]:
data_type = "node_features"
NEW_CLOUD_STORAGE_BUCKET = f'gs://ego_networks/twitter/{data_type}'

data = dd.read_csv(f"{CLOUD_STORAGE_BUCKET}/data/{data_type}*.csv", dtype={"withheld": "object"}).compute()
data = data.drop(columns="withheld").drop_duplicates()

print(data.shape)
# print(data.user.nunique(), data.following.nunique())
data.head()

(162771, 10)


,id,name,profile_image_url,public_metrics,username,verified,followers_count,following_count,tweet_count,listed_count
0,794433401591693312,Aran Komatsuzaki,https://pbs.twimg.com/profile_images/150798283...,"{'followers_count': 16111, 'following_count': ...",arankomatsuzaki,False,16111.0,72.0,1865.0,210.0
1,2509504696,Jericho Brown,https://pbs.twimg.com/profile_images/111968593...,"{'followers_count': 40468, 'following_count': ...",jerichobrown,True,40468.0,708.0,118856.0,408.0
2,1938030980,Angel Wicky,https://pbs.twimg.com/profile_images/144139505...,"{'followers_count': 355162, 'following_count':...",Angel_Wicky_II,True,355162.0,410.0,15435.0,1119.0
3,335544910,AC Justice Project,https://pbs.twimg.com/profile_images/145415812...,"{'followers_count': 1049, 'following_count': 4...",ACJProject,False,1049.0,497.0,3425.0,39.0
4,2622261,Brian Ulicny,https://pbs.twimg.com/profile_images/730781652...,"{'followers_count': 796, 'following_count': 24...",bulicny,False,796.0,2459.0,4624.0,68.0


In [12]:
data.id.value_counts()

1427408443296657408    9
794433401591693312     1
130978857              1
153130011              1
822121500              1
                      ..
161988348              1
133152680              1
19906690               1
246103                 1
111394815              1
Name: id, Length: 162763, dtype: int64

In [13]:
data_n = data[~(data.id.isin([1427408443296657408, 1120633726478823425]))]

In [16]:
dd.to_parquet(dd.from_pandas(data_n, npartitions = 10), 
              path = NEW_CLOUD_STORAGE_BUCKET,
              append = False,
              overwrite = True,
             ignore_divisions=True)

# dd.to_parquet(dd.from_pandas(data, npartitions = 10), 
#               path = NEW_CLOUD_STORAGE_BUCKET,
#               append = True,
#               overwrite = False,
#              ignore_divisions=True)

data_n = dd.read_parquet(
              path = NEW_CLOUD_STORAGE_BUCKET).compute()
data_n.head()

,id,name,profile_image_url,public_metrics,username,verified,followers_count,following_count,tweet_count,listed_count
0,794433401591693312,Aran Komatsuzaki,https://pbs.twimg.com/profile_images/150798283...,"{'followers_count': 16111, 'following_count': ...",arankomatsuzaki,False,16111.0,72.0,1865.0,210.0
0,746058182582206464,Peter C. Baker,https://pbs.twimg.com/profile_images/151109864...,"{'followers_count': 1093, 'following_count': 7...",apcbapcb,False,NaN,NaN,NaN,NaN
0,989084476088209409,Samarth Bansal,https://pbs.twimg.com/profile_images/156630914...,"{'followers_count': 2754, 'following_count': 1...",PySamarth,False,NaN,NaN,NaN,NaN
0,795701305691910144,Kevin Yang 楊凱筌,https://pbs.twimg.com/profile_images/130340944...,"{'followers_count': 6534, 'following_count': 3...",KevinKaichuang,False,NaN,NaN,NaN,NaN
0,880540913201577984,hazy,https://pbs.twimg.com/profile_images/151313100...,"{'followers_count': 365, 'following_count': 35...",FauciFan1958,False,NaN,NaN,NaN,NaN


In [19]:
data_type = "node_centrality"
file = f'gs://graph_data_science/twitter/data/processed/{data_type}.csv'

data = dd.read_csv(file).compute()

print(data.shape)
data.head()

(46, 9)


,user,cen_ev,cen_dg,cen_pg,rank_cen_ev,rank_cen_dg,rank_cen_pg,rank_combined,username
0,116994659,0.071912,0.000428,0.000009,1.000000,0.978261,0.956522,0.978100,['Sam Harris:SamHarrisOrg']
1,44196397,0.060478,0.000620,0.000010,0.913043,1.000000,1.000000,0.970131,['Elon Musk:elonmusk']
2,18208354,0.062272,0.000402,0.000009,0.934783,0.956522,0.760870,0.879506,['Joe Rogan:joerogan']
3,106204123,0.069367,0.000367,0.000009,0.978261,0.913043,0.717391,0.862119,['Jonathan Haidt:JonHaidt']
4,95092020,0.054365,0.000314,0.000009,0.826087,0.804348,0.913043,0.846552,['Dr Jordan B Peterson:jordanbpeterson']


In [41]:
data_type = "node_measures"
file = f'gs://graph_data_science/twitter/data/processed/measures/2/{data_type}.csv'

data = dd.read_csv(file).compute()

print(data.shape)
data.head()

(1206648, 3)


,measure_name,node,measure_value
0,pagerank,1210565171107815429,0.0
1,pagerank,1281443510,0.0
2,pagerank,632966383,0.0
3,pagerank,27631809,0.0
4,pagerank,14922097,0.0


In [42]:
data_n = data[~(data.node.isin([1427408443296657408, 1120633726478823425]))]

In [34]:
data_type = "node"
file = f'gs://ego_networks/twitter/measures/{data_type}/2'
dd.to_parquet(dd.from_pandas(data_n, npartitions = 10), 
              path = file,
              append = False,
              overwrite = True,
             ignore_divisions=True)

(None,)

## New Object

In [99]:
class DataConfig:
    root_dir = "gs://ego_networks/twitter" #os.getenv("CLOUD_STORAGE_BUCKET")
    file_paths = {
            "ties": f"{root_dir}/ties",
            "node_features": f"{root_dir}/node_features",
            "node_measures": f"{root_dir}/measures/node/3",
        }

class DataReader(DataConfig):
    def __init__(self, data_type):
        super().__init__()
        self.data_type = data_type
        
    def __preprocess(self, data):
        if self.data_type == "ties":
            data.following = data.following.apply(ast.literal_eval)
            data = data.explode("following")
            return data.dropna()
        elif self.data_type == "node_features":
            return data.drop_duplicates()
        else:
            return data
    
    def run(self):
        try:
            data = dd.read_parquet(path = self.file_paths.get(self.data_type)).compute()
            # logger.info(f"Read successful: {self.data_type}")
            return self.__preprocess(data)
        except Exception as error:
            # logger.error(f"Read unsuccessful: {data_type}, {error}")
            return pd.DataFrame()
        
class DataWriter(DataConfig):
    def __init__(self, data, data_type):
        super().__init__()
        self.data = data
        self.data_type = data_type
        
    def run(self, append = False, npartitions = 10):
        # logger.info(f"Writing {data_type}: {data.shape}")
        dd.to_parquet(dd.from_pandas(self.data, npartitions = 10), 
              path = self.file_paths.get(self.data_type),
              append = append,
              overwrite = not(append),
             ignore_divisions=True)

In [96]:
dw = DataWriter(data = data_n, data_type = 'node_measures')
dw.run(append = False)

In [101]:
for d in ["ties", "node_measures", "node_features"]:
    print(DataReader(data_type = d).run().head())

                  user   following
0  1210565171107815429  1281443510
0  1210565171107815429   632966383
0  1210565171107815429    27631809
0  1210565171107815429    14922097
0  1210565171107815429   228132689
  measure_name                 node  measure_value
0     pagerank  1210565171107815429            0.0
1     pagerank           1281443510            0.0
2     pagerank            632966383            0.0
3     pagerank             27631809            0.0
4     pagerank             14922097            0.0
                   id              name  \
0  794433401591693312  Aran Komatsuzaki   
0  746058182582206464    Peter C. Baker   
0  989084476088209409    Samarth Bansal   
0  795701305691910144    Kevin Yang 楊凱筌   
0  880540913201577984              hazy   

                                   profile_image_url  \
0  https://pbs.twimg.com/profile_images/150798283...   
0  https://pbs.twimg.com/profile_images/151109864...   
0  https://pbs.twimg.com/profile_images/156630914...   
0